In [1]:
# %% General imports
import numpy as np
import torch

# DeepMoD stuff
from deepymod.data import Dataset
from deepymod.data.burgers import BurgersDelta

%load_ext autoreload
%autoreload 2

In [2]:
# Settings
#if torch.cuda.is_available():
#    device = 'cuda'
#else:
#    device = 'cpu'

device = 'cpu'

# Settings for reproducibility
np.random.seed(42)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Making dataset

In [5]:
# Making dataset
v = 0.1
A = 1.0

x = np.linspace(-3, 4, 40)
t = np.linspace(0.1, 5.0, 25)
x_grid, t_grid = np.meshgrid(x, t, indexing='ij')
dataset = Dataset(BurgersDelta, v=v, A=A)
X, y = dataset.create_dataset(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1), n_samples=0, noise=0.2, random=True, normalize=True)
X, y = X.to(device), y.to(device)

# Bayes Training function

In [112]:
X = torch.tensor(dataset.library(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1), poly_order=2, deriv_order=3), dtype=torch.float32)
y = torch.tensor(dataset.time_deriv(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1)), dtype=torch.float32)

mask = torch.zeros(X.shape[1], dtype=torch.bool)
mask[[2, 5]] = 1

In [113]:
a = torch.nn.Parameter(torch.zeros(12))
b = torch.nn.Parameter(-torch.log(torch.var(y)))
optimizer = torch.optim.Adam([a, b], betas=(0.99, 0.99), amsgrad=True, lr=2e-3) # Defining optimizer

In [114]:
for epoch in np.arange(1e4):
    alpha_ = torch.exp(a).clamp(max=1e8)
    beta_ = torch.exp(b).clamp(max=1e8)
    
    loss = -SBL(X, y, mask, alpha_, beta_)[0]
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [115]:
alpha_

tensor([1.0000, 1.0000, 0.0234, 1.0000, 1.0000, 0.0175, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000], grad_fn=<ClampBackward>)

In [116]:
SBL(X, y, mask, alpha_, beta_)[1]

tensor([[  0.0000],
        [  0.0000],
        [ 15.5241],
        [  0.0000],
        [  0.0000],
        [-21.2503],
        [  0.0000],
        [  0.0000],
        [  0.0000],
        [  0.0000],
        [  0.0000],
        [  0.0000]], grad_fn=<MaskedScatterBackward>)

In [ ]:
beta_

In [93]:
def SBL(X, y, mask, alpha_, beta_):
    n_samples = X.shape[0]
    
    # Calculating coeff vectors
    X_normed = X[:, mask] / torch.norm(X[:, mask], dim=0, keepdim=True)
    alpha = alpha_[mask]
    
    A_inv = torch.inverse(torch.diag(alpha) + beta_ * X_normed.T @ X_normed)
    mn = beta_ * A_inv @ X_normed.T @ y
    
    # Calculating likelihood
    p = (
        - beta_ * torch.sum((y - X_normed @ mn) ** 2) 
        - torch.sum(alpha[:, None] * mn ** 2)
        + n_samples * torch.log(beta_)
        + torch.sum(torch.log(alpha))
        + torch.logdet(A_inv)
    )
    
    # Putting 0's in right spot
    coeffs =  torch.zeros((mask.shape[0], 1)).to(X.device).masked_scatter_(mask[:, None], mn)
    
    return p, coeffs